In [75]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [83]:
# Hyper-parameters 
input_size = 753
hidden_size = 500
num_classes = 2
num_epochs = 5
batch_size = 10
learning_rate = 0.01

In [84]:
# Define a customized class for spambase data
class ParkinsonDataset(Dataset):
    """Customized class that pre-processes parkinson dataset"""
    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Name of dataset file
        """
        # Read in data
        self.main = pd.read_csv(csv_file)
        
        # Standardize data
        m = np.mean(self.main.iloc[:,1:754].values)
        st = np.std(self.main.iloc[:,1:754].values)
        self.main.iloc[:,1:754] = (self.main.iloc[:,1:754]-m)/st

    def __len__(self):
        return len(self.main)

    def __getitem__(self, idx):
        # Get the features and make it into tensor
        features = self.main.iloc[idx, 1:754].values
        features = torch.from_numpy(features)
        # Get the label
        label = self.main.iloc[idx, -1]
        # Combine features and label into a tuple
        sample = (features,label)
        return sample

In [85]:
# Get dataset
dataset = ParkinsonDataset("pd_speech_features.csv")

# Specify portion for splitting, shuffle data, set seed
test_split = .2
shuffle_dataset = True
random_seed= 5

# Creating data indices for training and testing splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating data samplers and loaders

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [86]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #CUDA is a GPU


In [87]:
# Neural Network Class 
class NeuralNet(nn.Module):
    """ Feedforward Neural Network with 2 hidden layers and ReLU activation function."""
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        torch.manual_seed(7) # For reproducibility
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [88]:
# Set model
model = NeuralNet(input_size, hidden_size, num_classes).double().to(device)

In [89]:
# Set loss function and optimize algorithm
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Set start time
start = time.time()


for epoch in range(num_epochs):
    train_loss, test_loss = [], []
    correct = 0
    total = 0
    # Train the model
    for i, (x, y) in enumerate(train_loader):  
        # Move tensors to the configured device
        x = x.to(device)
        y = y.to(device)
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    
    with torch.no_grad():
        
        # Predict the model
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            lose = criterion(outputs, y)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
            test_loss.append(lose.item())

        print ("Epoch:", epoch + 1, ", Training Loss: ", np.mean(train_loss), ", Test loss: ", np.mean(test_loss))

# Set end time        
end = time.time()  
print('Accuracy is: {} %'.format(100 * correct / total))
print('It takes ' +  str(end - start)  + ' seconds to run Feedforward Neural Network algorithm on parkinson dataset.')

Epoch: 1 , Training Loss:  0.7084667956213714 , Test loss:  0.5897070544830928
Epoch: 2 , Training Loss:  0.5634390003914759 , Test loss:  0.5887524917866995
Epoch: 3 , Training Loss:  0.5596909758507567 , Test loss:  0.5924087495179351
Epoch: 4 , Training Loss:  0.5621637636871739 , Test loss:  0.5894135474505721
Epoch: 5 , Training Loss:  0.5677705401736718 , Test loss:  0.5899539104165603
Accuracy is: 70.86092715231788 %
It takes 10.414116144180298 seconds to run Feedforward Neural Network algorithm on parkinson dataset.
